# Compression

NOTE: Doesn't work

In this notebook we show the applications of the Discrete Resonance Spectrogram for compression purposes.

## 1. Imports

First we import the library of functions.

In [1]:
from utilities_store import *


## 2. Obtain spectrogram of the signal

First we obtain the spectrogram of the signal.

In [2]:
spectrogram, signal = get_spectrogram('nine.wav', N = 800, step_size = 750, 
                                          power_threshold = 1e-9, amp_threshold = 1e-8)

/Users/nastysushi/Mirror/_MULTIMEDIA/THESIS/thesis/github/fpt/fpt.py:224: RuntimeWarning: overflow encountered in power
  Z = np.power(self.zs.reshape(-1, 1), exponent)
/Users/nastysushi/Mirror/_MULTIMEDIA/THESIS/thesis/github/fpt/fpt.py:234: RuntimeWarning: invalid value encountered in matmul
  d = np.divide(Z @ ps[1:].reshape(-1, 1),
/Users/nastysushi/Mirror/_MULTIMEDIA/THESIS/thesis/github/fpt/fpt.py:235: RuntimeWarning: overflow encountered in matmul
  Z @ (qs[1:] * i).reshape(-1, 1))
/Users/nastysushi/Mirror/_MULTIMEDIA/THESIS/thesis/github/fpt/fpt.py:235: RuntimeWarning: invalid value encountered in matmul
  Z @ (qs[1:] * i).reshape(-1, 1))
/Users/nastysushi/Mirror/_MULTIMEDIA/THESIS/thesis/github/fpt/fpt.py:234: RuntimeWarning: invalid value encountered in divide
  d = np.divide(Z @ ps[1:].reshape(-1, 1),
/Users/nastysushi/Mirror/_MULTIMEDIA/THESIS/thesis/github/fpt/resonance.py:216: RuntimeWarning: overflow encountered in cdouble_scalars
  d_ = self.d * self.z ** (self.N - 1)


Progress: 0.00%
Progress: 2.29%
Progress: 4.59%
Progress: 6.88%
Progress: 9.17%
Progress: 11.46%
Progress: 13.76%
Progress: 16.05%
Progress: 18.34%
Progress: 20.64%
Progress: 22.93%
Progress: 25.22%
Progress: 27.51%
Progress: 29.81%
Progress: 32.10%
Progress: 34.39%
Progress: 36.69%
Progress: 38.98%
Progress: 41.27%
Progress: 43.56%
Progress: 45.86%
Progress: 48.15%
Progress: 50.44%
Progress: 52.74%


/Users/nastysushi/Mirror/_MULTIMEDIA/THESIS/thesis/github/fpt/resonance.py:216: RuntimeWarning: invalid value encountered in cdouble_scalars
  d_ = self.d * self.z ** (self.N - 1)
/Users/nastysushi/Mirror/_MULTIMEDIA/THESIS/thesis/github/fpt/resonance.py:266: RuntimeWarning: invalid value encountered in cdouble_scalars
  return replace(self, d=self.d * self.z ** offset)


Progress: 55.03%
Progress: 57.32%
Progress: 59.61%
Progress: 61.91%
Progress: 64.20%
Progress: 66.49%
Progress: 68.79%
Progress: 71.08%
Progress: 73.37%
Progress: 75.66%
Progress: 77.96%
Progress: 80.25%
Progress: 82.54%
Progress: 84.84%
Progress: 87.13%
Progress: 89.42%
Progress: 91.72%
Progress: 94.01%
Progress: 96.30%
Progress: 98.59%


## 3. Compress spectrogram

In order to compress the spectrogram we will filter low power resonances and change the data type of the resonance coefficients from complex128 to complex64. Then we compare the size in bytes of the spectrogram with the original signal.

In [9]:
ResonanceSet(np.array([hash(Resonance(res.d.astype('complex64'), res.w.astype('complex64'), 
                                                              res.z.astype('complex64'), res.N, res.sample_rate)) 
                                                                for res in spectrogram.elements]),
                                            np.array(spectrogram.onsets))

ResonanceSet(elements=array([-6843574113805233475,  -263601363406802399,  8901061906405070399,
       ...,  3139832992831693347,   430224505105016130,
        4575256976056509452]), onsets=array([    0,     0,     0, ..., 32250, 32250, 32250]))

In [45]:
# Original code that didn't work
# casted_spectrogram = ResonanceSpectrogram(np.array([hash(Resonance(res.d.astype('complex64'), res.w.astype('complex64'), 
#                                                               res.z.astype('complex64'), res.N, res.sample_rate)) 
#                                                                 for res in spectrogram.elements]),
#                                             np.array(spectrogram.onsets))

#filtered_spectrogram = casted_spectrogram.filter(lambda onset, res: res.power > 5e-4 or abs(res.decay) >= 1000)


filtered_spectrogram = spectrogram.filter(lambda onset_other, res_other: res_other.power > 5e-4 or abs(res_other.decay) >= 1000)
filtered_spectrogram

ResonanceSet(elements=array([Resonance(d=(-0.000987851385908302+0.00028690285773348104j), w=(-1339.7971053901488+269.90297951798743j), z=(1.0056747193825453+0.030562693433540976j), N=750, sample_rate=44100),
       Resonance(d=(-0.000987851385908302-0.00028690285773348104j), w=(1339.7971053901488+269.90297951798743j), z=(1.0056747193825453-0.030562693433540976j), N=750, sample_rate=44100),
       Resonance(d=(0.08247369974853426+0.05513686089121046j), w=(1597.1244116397506-192.66682386233003j), z=(0.9949877992306129-0.03605021381725134j), N=750, sample_rate=44100),
       Resonance(d=(0.08247369974853426-0.05513686089121046j), w=(-1597.1244116397506-192.66682386233003j), z=(0.9949877992306129+0.03605021381725134j), N=750, sample_rate=44100),
       Resonance(d=(0.006722769732016562-0.0189608906642069j), w=(-2575.5735814240925+31.48128631147162j), z=(0.9990079264861951+0.05841151628135959j), N=750, sample_rate=44100),
       Resonance(d=(0.006722769732016562+0.0189608906642069j), w=(257

##### Original signal

Size of the original signal in kB:

(number of samples * 16 bits / sample) / (8 bits / byte * 1000 Bytes / 1kB)

In [46]:
print("Size of the original signal in kB:")
len(signal) * 16 / (8 * 1000)

Size of the original signal in kB:


65.42

##### Filtered spectrogram

Size of the filtered spectrogram in kB:

(number of resonances * 2 coeff / resonance * 64 bits per coeff) / (2 (mirrored resonances) * 8 bits / byte * 1000 Bytes / 1 kB)

In [47]:
print("Size of the filtered spectrogram in kB:")
(len(filtered_spectrogram.elements) * 2 * 64) / (2 * 8 * 1000)

Size of the filtered spectrogram in kB:


2.408

##### Compression rate:

1 / (Size of the filtered spectrogram / size of the orginal signal)

In [48]:
print("Compression rate:")
1 / (((len(filtered_spectrogram.elements) * 2 * 64) / (2 * 8 * 1000)) / (len(signal) * 16 / (8 * 1000)))

Compression rate:


27.167774086378742

Reproduce the original audio signal and the time reconstruction from the filtered spectrogram

Original audio signal

In [49]:
ipd.Audio(data = signal, rate = spectrogram.sample_rate[0])

Time recosntruction of the filtered spectrogram

In [51]:
ipd.Audio(data = filtered_spectrogram.reconstruction, rate = spectrogram.sample_rate[0])

# DOESN'T WORK

# 4. Conclusions

In the case of window of analysis (N) equals to step length (step_size) (no overlap between windows), we obtain K/2 resonances for a signal of K samples. This means that we obtain K complex coefficients. This is the configuration where the number of generated coefficients is the lowest. However due to the non-overlapping, this coefficients can't be cast to complex64 from complex128 due to the high values they reach. If we cast this coefficients we can hear some arctifacts in the reconstruction.

However we can solve this problem by adding a small overlap between windows (N > step_size). This way the number of cofficients generated is bigger than the number of samples of the original signal, however the values of these coefficients can be casted to complex64 from complex128 without producing artifacts in the reconstruction of the signal. After filtering low power resonances we can achieve compression rates > 1.

To have a perspective point about the compression rates achieved by MP3, it is worth to mention that for the audio signal shown in the example, MP3 can reach a compression rate of around 10 when configured for the lowest quality (no perceptual signal degradation)